In [1]:
import os
import sys
import pandas
import numpy

In [2]:
CURR_DIR = os.getcwd()
PROJECT_DIR = os.path.dirname(CURR_DIR)
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
DATA_MET = os.path.join(DATA_DIR, 'MET')
DATA_RAW = os.path.join(DATA_DIR, 'raw')
TOKEN_MET = os.path.join(DATA_DIR, 'MET', 'token.txt')

GEN_DB = os.path.join(DATA_RAW, 'general.db')

In [3]:
MET_URL = "https://api.met.gov.my/v2"
MET_URL2 = "https://api.met.gov.my/v2/data"
DATA_TYPE_URL = "https://api.met.gov.my/v2/datatypes"
LOCATION_BASE_URL = "https://api.met.gov.my/v2.1/locations?locationcategoryid="

In [4]:
sys.path.append(PROJECT_DIR)

In [5]:
from src.MET import getDataMET, getInfoMET

In [6]:
TOKEN = getDataMET.read_text_file(TOKEN_MET)

In [7]:
# LOCATIONS = ['STATE', 'DISTRICT', 'TOWN', 'TOURISTDEST', 'WATERS']
# getInfoMET.getCSV(DATA_TYPE_URL, LOCATION_BASE_URL, LOCATIONS, DATA_MET, TOKEN)

In [8]:
getJSON = getDataMET.get_met_api_data(
    "FORECAST", "LOCATION:13", "2024-01-01", "2024-01-01", MET_URL2, TOKEN
)

In [9]:
df = getDataMET.extract_weather_info(getJSON)

In [10]:
df

data_type,location_name,date,FGA,FGM,FGN,FMAXT,FMINT,FSIGW
0,SABAH,2024-01-01T00:00:00+08:00,Isolated thunderstorms over inland areas,No rain,Isolated rain over inland areas,34,22,Isolated thunderstorms over inland areas


In [11]:
import sqlite3

conn = sqlite3.connect(GEN_DB)

In [12]:
create_sql = "CREATE TABLE IF NOT EXISTS forecast (data_type INTEGER, location_name TEXT, date TEXT, FGA TEXT, FGM TEXT, FGN TEXT, FMAXT INTEGER, FMINT INTEGER, FSIGW text)"
cursor = conn.cursor()
cursor.execute(create_sql)

In [14]:
for row in df.itertuples():
    insert_sql = f"INSERT INTO forecast (data_type, location_name, date, FGA, FGM, FGN, FMAXT, FMINT, FSIGW) \
        VALUES ({row[0]}, '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', {row[6]}, {row[7]}, '{row[8]}')"
    cursor.execute(insert_sql)

In [15]:
conn.commit()